# Rule Based Classification (Client Segmentation)

**Business Need:**
A game company wants to create client segments based on price information.  
So the company can estimates the average revenue from each client segment using this basic model.
E.g.: How much does this company earn from each type of personas in average? (from a 25 year old male client in Brazil?)

"Persona" dataset contains price and clients' demographic information. Each row represents a sale record.

**Variables:**

* PRICE – How much client spent on
* SOURCE – Type of operation system
* SEX – Client's gender
* COUNTRY – Client's country
* AGE – Client's age

**Section 1 : Describe the data (basic information)**

In [1]:
import numpy as np
import pandas as pd

In [2]:
df = pd.read_csv("../input/persona/week_02_data.csv")

In [3]:
df.head() # top 5 rows

,PRICE,SOURCE,SEX,COUNTRY,AGE
0,39,android,male,bra,17
1,39,android,male,bra,17
2,49,android,male,bra,17
3,29,android,male,tur,17
4,49,android,male,tur,17


In [4]:
df.tail() # latest 5 rows

,PRICE,SOURCE,SEX,COUNTRY,AGE
4995,29,android,female,bra,31
4996,29,android,female,bra,31
4997,29,android,female,bra,31
4998,39,android,female,bra,31
4999,29,android,female,bra,31


In [5]:
df.shape # shape of the data (n.of cases and variables)

(5000, 5)

In [6]:
df.info() # each variables and their data types

<class 'pandas.core.frame.DataFrame'>
Int64Index: 5000 entries, 0 to 4999
Data columns (total 5 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   PRICE    5000 non-null   int64 
 1   SOURCE   5000 non-null   object
 2   SEX      5000 non-null   object
 3   COUNTRY  5000 non-null   object
 4   AGE      5000 non-null   int64 
dtypes: int64(2), object(3)
memory usage: 234.4+ KB


In [7]:
df.columns # name of columns

Index(['PRICE', 'SOURCE', 'SEX', 'COUNTRY', 'AGE'], dtype='object')

In [8]:
df.index # index numbers of each case in the data

Int64Index([   0,    1,    2,    3,    4,    5,    6,    7,    8,    9,
            ...
            4990, 4991, 4992, 4993, 4994, 4995, 4996, 4997, 4998, 4999],
           dtype='int64', length=5000)

In [9]:
df.describe().T # descriptive information of numerical variables. 
# basic description of price and age

,count,mean,std,min,25%,50%,75%,max
PRICE,5000.0,34.1320,12.464897,9.0,29.0,39.0,39.0,59.0
AGE,5000.0,23.5814,8.995908,15.0,17.0,21.0,27.0,66.0


In [10]:
df.isnull().values.any() # Is there any null values?
# No missing values

False

**Section 2 : Number of unique sources and their frequencies.**

In [11]:
df["SOURCE"].nunique() # there are 2 unique sources

2

In [12]:
df.groupby(["SOURCE"]).agg({"SOURCE":
                                "count"})

,SOURCE
SOURCE,
android,2974
ios,2026


**Section 3: Number of unique Prices.**

In [13]:
df["PRICE"].nunique() # 6 different prices are applied

6

**Section 4: Count of each unique PRICE**

In [14]:
df.groupby(["PRICE"]).agg({"PRICE":
                                "count"})

,PRICE
PRICE,
9,200
19,992
29,1305
39,1260
49,1031
59,212


**Section 5: N. of cases by each COUNTRY**

In [15]:
df.groupby(["COUNTRY"]).agg({"COUNTRY":
                                "count"})

,COUNTRY
COUNTRY,
bra,1496
can,230
deu,455
fra,303
tur,451
usa,2065


**Section 6: What is the total revenue by COUNTRIES?**

In [16]:
df.groupby(["COUNTRY"]).agg({"PRICE":
                                "sum"})

,PRICE
COUNTRY,
bra,51354
can,7730
deu,15485
fra,10177
tur,15689
usa,70225


**Section 7: N. of cases by each SOURCE (operating systems)**

In [17]:
df.groupby(["SOURCE"]).agg({"SOURCE":
                                "count"})

,SOURCE
SOURCE,
android,2974
ios,2026


**Section 8: Average PRICE by countries**

In [18]:
df[["COUNTRY","PRICE"]].groupby("COUNTRY").mean()

,PRICE
COUNTRY,
bra,34.327540
can,33.608696
deu,34.032967
fra,33.587459
tur,34.787140
usa,34.007264


**Section 9: Average PRICE by sources**

In [19]:
df.pivot_table("PRICE", index=["SOURCE"])

,PRICE
SOURCE,
android,34.174849
ios,34.069102


**Section 10: Average PRICE by COUNTRY-SOURCE differentiation**

In [20]:
df.pivot_table(values="PRICE", index=["COUNTRY", "SOURCE"], aggfunc='mean')

PRICE
COUNTRY SOURCE            
bra     android  34.387029
        ios      34.222222
can     android  33.330709
        ios      33.951456
deu     android  33.869888
        ios      34.268817
fra     android  34.312500
        ios      32.776224
tur     android  36.229437
        ios      33.272727
usa     android  33.760357
        ios      34.371703

**Section 11: Average revenue by COUNTRY, SOURCE, SEX, AGE.**

In [21]:
df.pivot_table(values="PRICE", index=["COUNTRY", "SOURCE","SEX","AGE"], aggfunc='mean')


PRICE
COUNTRY SOURCE  SEX    AGE           
bra     android female 15   38.714286
                       16   35.944444
                       17   35.666667
                       18   32.255814
                       19   35.206897
...                               ...
usa     ios     male   42   30.250000
                       50   39.000000
                       53   34.000000
                       55   29.000000
                       59   46.500000

[348 rows x 1 columns]

In [22]:
df.groupby(["COUNTRY", "SOURCE","SEX","AGE"]).agg({"PRICE":
                                                   "mean"})

PRICE
COUNTRY SOURCE  SEX    AGE           
bra     android female 15   38.714286
                       16   35.944444
                       17   35.666667
                       18   32.255814
                       19   35.206897
...                               ...
usa     ios     male   42   30.250000
                       50   39.000000
                       53   34.000000
                       55   29.000000
                       59   46.500000

[348 rows x 1 columns]

**CREATING PERSONAS**
**Section 12: Let's see prices sorted descending**

In [23]:
agg_df = df.pivot_table("PRICE", index=["COUNTRY", "SOURCE","SEX","AGE"], aggfunc='mean').sort_values("PRICE", ascending=False)

In [24]:
agg_df.head()

,,,,PRICE
COUNTRY,SOURCE,SEX,AGE,
bra,android,male,46,59.0
usa,android,male,36,59.0
fra,android,female,24,59.0
usa,ios,male,32,54.0
deu,android,female,36,49.0


**Section 13: Changing Index Names into Column Name**

In [25]:
agg_df.index.names

FrozenList(['COUNTRY', 'SOURCE', 'SEX', 'AGE'])

In [26]:
agg_df.reset_index(inplace=True)

In [27]:
agg_df.index.names

FrozenList([None])

In [28]:
agg_df.head() # now we have rearranged the column names.

,COUNTRY,SOURCE,SEX,AGE,PRICE
0,bra,android,male,46,59.0
1,usa,android,male,36,59.0
2,fra,android,female,24,59.0
3,usa,ios,male,32,54.0
4,deu,android,female,36,49.0


**Section 14: Adding Age Categories**

In [29]:
agg_df["AGE_CAT"] = pd.cut(agg_df["AGE"], bins=[0, 18, 23, 30, 40, 70],  labels=['0_18', '19_23', '24_30', '31_40', '41_70'])

In [30]:
agg_df.head()

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT
0,bra,android,male,46,59.0,41_70
1,usa,android,male,36,59.0,31_40
2,fra,android,female,24,59.0,24_30
3,usa,ios,male,32,54.0,31_40
4,deu,android,female,36,49.0,31_40


**Section 15: Creating personas for each user**

In [31]:
def create_persona(dataframe, new_col_name):
    dataframe[new_col_name] = dataframe.loc[:,["COUNTRY", "SOURCE", "SEX", "AGE_CAT"]] \
        .apply(lambda x: "_".join(x.str.upper()), axis=1)

    return dataframe.groupby([new_col_name])["PRICE"].mean().reset_index().sort_values(by="PRICE", ascending=False)

In [32]:
create_persona(agg_df, "customer_levels")

,customer_levels,PRICE
54,FRA_ANDROID_FEMALE_24_30,45.428571
86,TUR_IOS_MALE_24_30,45.000000
87,TUR_IOS_MALE_31_40,42.333333
72,TUR_ANDROID_FEMALE_31_40,41.833333
25,CAN_ANDROID_MALE_19_23,40.111111
...,...,...
88,TUR_IOS_MALE_41_70,25.052632
85,TUR_IOS_MALE_19_23,24.833333
56,FRA_ANDROID_MALE_0_18,24.625000
22,CAN_ANDROID_FEMALE_24_30,19.000000


**Section 16: Creating Segments based on Prices**

In [33]:
agg_df["SEGMENT"] = pd.qcut(agg_df["PRICE"], 4, labels = ["D", "C", "B", "A"]) # (A: best, D: worst)

Now we have the rule-based classified data

In [34]:
agg_df.head() # attention to PRICE and SEGMENT group 

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT,customer_levels,SEGMENT
0,bra,android,male,46,59.0,41_70,BRA_ANDROID_MALE_41_70,A
1,usa,android,male,36,59.0,31_40,USA_ANDROID_MALE_31_40,A
2,fra,android,female,24,59.0,24_30,FRA_ANDROID_FEMALE_24_30,A
3,usa,ios,male,32,54.0,31_40,USA_IOS_MALE_31_40,A
4,deu,android,female,36,49.0,31_40,DEU_ANDROID_FEMALE_31_40,A


In [35]:
agg_df.tail() # attention to PRICE and SEGMENT group 

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT,customer_levels,SEGMENT
343,usa,ios,female,38,19.0,31_40,USA_IOS_FEMALE_31_40,D
344,usa,ios,female,30,19.0,24_30,USA_IOS_FEMALE_24_30,D
345,can,android,female,27,19.0,24_30,CAN_ANDROID_FEMALE_24_30,D
346,fra,android,male,18,19.0,0_18,FRA_ANDROID_MALE_0_18,D
347,deu,android,male,26,9.0,24_30,DEU_ANDROID_MALE_24_30,D


Let's see the price information among client segments

In [36]:
agg_df.groupby("SEGMENT").agg({"PRICE": ["min", "max", "mean","sum"]}) # differences among client segments

PRICE                                   
               min        max       mean          sum
SEGMENT                                              
D         9.000000  31.105263  27.302596  2375.325850
C        31.173913  34.000000  32.933339  3128.667165
B        34.185185  37.000000  35.436170  2870.329792
A        37.095238  59.000000  41.434736  3521.952577

Let's see the price information among personas

In [37]:
agg_df.groupby("customer_levels").agg({"PRICE": ["min", "max", "mean","sum"]}) # differences among personas

PRICE                                  
                                min        max       mean         sum
customer_levels                                                      
BRA_ANDROID_FEMALE_0_18   32.255814  38.714286  35.645303  142.581211
BRA_ANDROID_FEMALE_19_23  31.000000  37.292683  34.077340  170.386702
BRA_ANDROID_FEMALE_24_30  22.333333  45.666667  33.863946  237.047619
BRA_ANDROID_FEMALE_31_40  24.000000  43.000000  34.898326  209.389959
BRA_ANDROID_FEMALE_41_70  32.333333  44.000000  36.737179  146.948718
...                             ...        ...        ...         ...
USA_IOS_MALE_0_18         32.913043  34.714286  33.983495  135.933982
USA_IOS_MALE_19_23        32.170732  38.534884  34.901872  104.705615
USA_IOS_MALE_24_30        24.000000  42.333333  34.838143  209.028860
USA_IOS_MALE_31_40        24.000000  54.000000  36.206324  181.031621
USA_IOS_MALE_41_70        29.000000  46.500000  35.750000  178.750000

[109 rows x 4 columns]

In [38]:
agg_df.groupby(["SEGMENT", "customer_levels"]).agg({"PRICE":
                                                   "mean"}) 
# differences among customer personas and segments in terms of average price

PRICE
SEGMENT customer_levels                    
D       BRA_ANDROID_FEMALE_0_18         NaN
        BRA_ANDROID_FEMALE_19_23  31.000000
        BRA_ANDROID_FEMALE_24_30  24.238095
        BRA_ANDROID_FEMALE_31_40  24.000000
        BRA_ANDROID_FEMALE_41_70        NaN
...                                     ...
A       USA_IOS_MALE_0_18               NaN
        USA_IOS_MALE_19_23        38.534884
        USA_IOS_MALE_24_30        42.030303
        USA_IOS_MALE_31_40        47.500000
        USA_IOS_MALE_41_70        42.750000

[436 rows x 1 columns]

**Section 17: Estimating Client Segment for New Users by Using Personas**

In [39]:
new_user = 'TUR_ANDROID_FEMALE_31_40' # A female Turkish client in age group 31-40 is in Segment A

In [40]:
agg_df[agg_df["customer_levels"] == new_user] 

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT,customer_levels,SEGMENT
18,tur,android,female,32,43.000000,31_40,TUR_ANDROID_FEMALE_31_40,A
35,tur,android,female,31,40.666667,31_40,TUR_ANDROID_FEMALE_31_40,A


In [41]:
new_user = 'FRA_IOS_FEMALE_31_40' # A female French client in age group 31-40 is in Segment C

In [42]:
agg_df[agg_df["customer_levels"] == new_user]

,COUNTRY,SOURCE,SEX,AGE,PRICE,AGE_CAT,customer_levels,SEGMENT
208,fra,ios,female,40,33.000000,31_40,FRA_IOS_FEMALE_31_40,C
221,fra,ios,female,31,32.636364,31_40,FRA_IOS_FEMALE_31_40,C


Reference: https://www.miuul.com/